## 문장 유사도(Text Similarity)를 이용한 ChatBot 실습
- 사전 훈련된 한국어 SBERT 모델 객체 생성 -> 문장 임베딩 벡터 생성
- 사용자 질문에 대한 응답 구현
- 사용자의 질문이 들어오면 해당 질문에 대한 문장 임베딩 벡터 생성
- 사용자 질문의 문장 임베딩 벡터 값과 챗봇 데이터의 임베딩 컬럼('embedding')에 저장해둔 모든 질문
- 샘플들의 문장 임베딩 벡터 값들을 이용, 코사인 유사도를 계산
- 코사인 유사도 값이 가장 높은 질문 샘플 검색
- 해당 질문 샘플과 짝이 되는 답변 샘플 반환

### 데이터 생성 및 함수 임폴트

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import linear_kernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf
from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk 
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
from sentence_transformers import SentenceTransformer


## 문장 임베딩 생성용 데이터 불러오기

In [2]:
### 챗봇 생성용 데이터 불러오기

file_path = 'D:\Code\Deep Learning/ChatBotData.csv'

df = pd.read_csv(file_path)

print(df)

                             Q                         A  label
0                       12시 땡!                하루가 또 가네요.      0
1                  1지망 학교 떨어졌어                 위로해 드립니다.      0
2                 3박4일 놀러가고 싶다               여행은 언제나 좋죠.      0
3              3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.      0
4                      PPL 심하네                눈살이 찌푸려지죠.      0
...                        ...                       ...    ...
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!      2
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.      2
11820              흑기사 해주는 짝남.                    설렜겠어요.      2
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.      2
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.      2

[11823 rows x 3 columns]


In [3]:
### 사전 학습된 한국어 SentenceBERT 모델 생성
model_name = 'ddobokki/klue-roberta-base-nli-sts'
model = SentenceTransformer(model_name)

In [4]:
### 모델 테스트 : 텍스트 -> 임베딩

sentence = ["본 발명은 바위수염 추출물을 포함하는 항비만용 조성물 및 그 제조방법에 관한 것이다"]

### model.encode(sentence) 사용

embedding = model.encode(sentence)

# 문장 임베딩 결과 확인
print(f'임베딩 벡터의 모양 확인 : \n{embedding.shape}')

print('*'*80)

print(f'첫번째 임베딩 벡터의 내용 확인 : \n{embedding}')

# 문장이 하나인데 대괄호에 안넣으면? 1차원이 됨. 하라는대로 대괄호를 위에 sentence에 넣으면 2차원 행렬이 됨

임베딩 벡터의 모양 확인 : 
(1, 768)
********************************************************************************
첫번째 임베딩 벡터의 내용 확인 : 
[[ 2.47432545e-01  3.19152832e-01 -2.05433086e-01  4.23581570e-01
  -5.40823638e-01  6.10795736e-01 -2.74022907e-01 -5.51382422e-01
  -2.76437491e-01 -1.81457639e-01  4.33313996e-01 -4.46095854e-01
   7.90904537e-02 -9.16764736e-01 -2.76828259e-01 -7.00665731e-03
   2.35826466e-02  6.20459974e-01  7.21695423e-01 -7.87704706e-01
  -3.69594634e-01  3.44950855e-01  4.21566725e-01 -4.31745797e-01
  -1.84948713e-01  1.36051178e-01  1.34623542e-01 -5.27529597e-01
  -2.24852368e-01 -4.65934604e-01  5.44648409e-01 -1.61633119e-01
   1.30852982e-01 -2.97314227e-01 -3.55800748e-01  3.07409726e-02
  -7.84511864e-01 -7.73454383e-02  1.87774405e-01  4.93741259e-02
   1.90536708e-01 -1.20568149e-01  3.50683361e-01  6.41438365e-02
   1.39915302e-01 -1.95609972e-01  9.60729942e-02  1.94002151e-01
   3.45328361e-01  2.39388183e-01 -4.80340809e-01  2.85571158e-01
   2.81684458e

In [5]:
### 문장 임베딩 생성
'''
### apply(lambda x : model.encode(x))
'''

# 임베딩 벡터 생성
embeddings = df.loc[:, 'Q'].apply(lambda x: model.encode(x)).values #values를 붙이면 2차원행렬이 1차원벡터가 됨.

# for문을 쓰지 않고도 컬럼에 있는 값들을 순차적으로 가져다 처리할 수 있다는게 장점이다!!!.

# 결과 확인하기
print(f'문장 임베딩 벡터 전체의 모양 : {embeddings.shape}')

print('*'*80)

print(f'전체 문장 임베딩 벡터 출력 : \n{embeddings}')

# 기존 df에 데이터프레임을 만든다. embedding

In [ ]:
### 저장된 임베딩 벡터 불러오기

# 파일 경로 설정하기
file_path = 'D:\Code\DataSets/train_embeddings.npy'

# np.load() 사용
embeddings = np.load(file_path, allow_pickle=True)

print(f' 결과값 : \n{embeddings}')

In [ ]:
### 챗봇 생성용 데이터 --> embedding이라는 새로운 컬럼 생성, 문장 임베딩 벡터 삽입
df['embedding'] = embeddings

# 결과 확인하기
print(f'전체 데이터프레임 확인 : \n{df}')
print('*'*80)

print(f'embedding 컬럼의 0번째 성분 원소 확인 : \n{df.iloc[0, -1]}')

# 새로운 데이터프레임이 만들어지면 추가된 컬럼은 항상 끝번가서 붙는다, 0번행에 끝번컬럼. 딱 하나의 데이터.


## ChatBot 생성

In [ ]:
### 두 문장의 텍스트 유사도 계산 함수 정의

# 필요한 함수 임폴트

from numpy import dot # 벡터의 내적, 행렬의 곱 
from numpy.linalg import norm # 피타고라스의정리 사용할 수 있게 해줌
from sklearn.metrics.pairwise import cosine_similarity

# 사용자 정의 함수|
def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))
#분자내적, 분모norm(피타고라스)

In [ ]:
### 사용자 질문에 대한 응답 구현
'''
1. 사용자의 질문이 들어오면 해당 질문에 대한 문장 임베딩 값을 구한다.
2. 사용자 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 컬럼(df.lco[:, 'embedding'])에
저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 이용, 코사인 유사도를 계산
--> 코사인 유사도 값을 저장하는 컬럼(df.loc[:, 'score'])을 생성하고 값을 저장
3. 코사인 유사도 값이 가장 높은 질문 샘플을 찾는다.
4. 해당 질문 샘플과 짝이 되는 답변 샘플을 반환한다.
'''

# 사용자 정의 함수 생성
def chatbot(question) : 
    # 사용자 질문 문장 --> 문장 임베딩 벡터 구하기
    embedding = model.encode(question) #텍스트를 그대로 받는다.
    #return embedding
	# 코사인 유사도 계산 --> score컬럼 생성, 코사인 유사도 값을 저장
    print(df.loc[:,'embedding'].apply(lambda x: cos_sim(x, embedding)).values)
	# df['score'] = df.loc[:,'embedding'].apply(lambda x: cos_sim(x, embedding)).values
	#각 Q에 해당하는 sentences들 786짜리 임베딩 벡터값들이 들어있다! 11123개.
	# vprint(df)



In [ ]:
text = input()

result = chatbot(text)

print(result)